In [2]:
import requests
import json
import pandas as pd
import time
from config import API_key
import logging

# Получение данных о книгах

In [ ]:
logging.basicConfig(level=logging.INFO, filename="API_google_log.log",filemode="w",
                    format="%(asctime)s %(levelname)s %(message)s")

In [ ]:
'''
with open('scraping_log.log', 'w'):
    pass # Clearing logs
'''

In [ ]:
all_books = pd.read_csv("All_added_books_data.csv").drop_dublicates()

In [ ]:
titles = all_books['Title'].dropna().tolist()

base_url = "https://www.googleapis.com/books/v1/volumes"
results = []

for title in titles:
    logging.info(f"Started getting info about '{title}' title")
    search_query = title.replace(' ', '+')
    url = f"{base_url}?q=intitle:{search_query}"
    flag = True
    while flag:
      try:
          response = requests.get(url, headers={'accept': 'application/json', 'X-API-KEY': API_key})
          if response.status_code == 200:
              flag = False
              response_data = response.json()

              if 'items' in response_data and len(response_data['items']) > 0:
                  first_result = response_data['items'][0]['volumeInfo']
                  results.append({
                      "Title": title,
                      "API Title": first_result.get('title'),
                      "Authors": first_result.get('authors', []),
                      "Publisher": first_result.get('publisher'),
                      "Published Date": first_result.get('publishedDate'),
                      "Description": first_result.get('description'),
                      "Page Count": first_result.get('pageCount'),
                      "Print Type": first_result.get('printType'),
                      "Categories": first_result.get('categories', []),
                      "Average Rating": first_result.get('averageRating'),
                      "Ratings Count": first_result.get('ratingsCount'),
                      "Language": first_result.get('language'),
                  })
              else:
                  results.append({
                      "Title": title,
                      "API Title": None,
                      "Authors": None,
                      "Publisher": None,
                      "Published Date": None,
                      "Description": None,
                      "Page Count": None,
                      "Print Type": None,
                      "Categories": None,
                      "Average Rating": None,
                      "Ratings Count": None,
                      "Language": None,
                  })
          else:
              logging.warning(f"Error appeared while requesting data for '{title}' title: {response.status_code}")
              time.sleep(1)
      except Exception as e:
          logging.error(f"Error appeared while requesting data for '{title}'", exc_info=e)
          break



In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("all_api_books.csv", index=False)

# Дополнительные эндпоинты



Данный запрос получает все полки данного пользователя.

In [1]:
userID = '104174585791801023522'

In [7]:
page_1 = requests.get(f'https://www.googleapis.com/books/v1/users/{userID}/bookshelves', headers ={'accept': 'application/json', 'X-API-KEY': API_key})
page_1

<Response [200]>

In [8]:
res_1 = page_1.json()

In [9]:
res_1.keys()

dict_keys(['kind', 'items'])

In [10]:
res_1

{'kind': 'books#bookshelves',
 'items': [{'kind': 'books#bookshelf',
   'id': 0,
   'title': 'Favorites',
   'access': 'PUBLIC',
   'updated': '2025-03-04T22:08:47.933Z',
   'created': '2025-03-04T22:08:47.933Z',
   'volumeCount': 0,
   'volumesLastUpdated': '2025-03-04T22:08:47.930Z'},
  {'kind': 'books#bookshelf',
   'id': 1001,
   'title': 'Study',
   'access': 'PUBLIC',
   'updated': '2025-03-04T22:08:49.829Z',
   'created': '2025-03-04T22:08:49.829Z',
   'volumeCount': 5,
   'volumesLastUpdated': '2025-03-04T22:08:49.828Z'}]}

In [11]:
df_1 = pd.DataFrame(res_1['items'])
df_1

,kind,id,title,access,updated,created,volumeCount,volumesLastUpdated
0,books#bookshelf,0,Favorites,PUBLIC,2025-03-04T22:08:47.933Z,2025-03-04T22:08:47.933Z,0,2025-03-04T22:08:47.930Z
1,books#bookshelf,1001,Study,PUBLIC,2025-03-04T22:08:49.829Z,2025-03-04T22:08:49.829Z,5,2025-03-04T22:08:49.828Z


In [33]:
df_1.to_csv("more_bookshelf_api.csv", index=False)

Следующий запрос выполняет запрос всех книг на определенной полке.

In [13]:
bookshelfID = '1001'

In [14]:
page_2 = requests.get(f'https://www.googleapis.com/books/v1/users/{userID}/bookshelves/{bookshelfID}/volumes', headers ={'accept': 'application/json', 'X-API-KEY': API_key})
page_2


<Response [200]>

In [16]:
res_2 = page_2.json()

In [17]:
res_2.keys()

dict_keys(['kind', 'totalItems', 'items'])

In [18]:
res_2

{'kind': 'books#volumes',
 'totalItems': 5,
 'items': [{'kind': 'books#volume',
   'id': 'LZ77DwAAQBAJ',
   'etag': 'GI9qujIBdiA',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/LZ77DwAAQBAJ',
   'volumeInfo': {'title': 'Legge Di Attrazione',
    'authors': ['Carolina Meli'],
    'publisher': 'Litres',
    'publishedDate': '2020-08-17',
    'description': 'Se te ne stai tranquillamente a casa e ti stai preparando per andare a dormire, fatti sempre la stessa domanda. Cerca di capire quali sono le cose che ti sembrano vuote e inutili, e quali invece ti mettono un brivido dentro. Fa in modo che il tuo ...',
    'industryIdentifiers': [{'type': 'ISBN_13', 'identifier': '9785042777189'},
     {'type': 'ISBN_10', 'identifier': '5042777180'}],
    'readingModes': {'text': True, 'image': True},
    'pageCount': 16,
    'printType': 'BOOK',
    'categories': ['Religion'],
    'maturityRating': 'NOT_MATURE',
    'allowAnonLogging': False,
    'contentVersion': '0.58.58.0.preview.3',

In [24]:
d_2 = res_2['items']

In [29]:
info_list = [item['volumeInfo'] for item in d_2]

In [30]:
df_2 = pd.DataFrame(info_list)
df_2

,title,authors,publisher,publishedDate,description,industryIdentifiers,readingModes,pageCount,printType,categories,maturityRating,allowAnonLogging,contentVersion,panelizationSummary,imageLinks,previewLink,infoLink,canonicalVolumeLink
0,Legge Di Attrazione,[Carolina Meli],Litres,2020-08-17,Se te ne stai tranquillamente a casa e ti stai...,"[{'type': 'ISBN_13', 'identifier': '9785042777...","{'text': True, 'image': True}",16,BOOK,[Religion],NOT_MATURE,False,0.58.58.0.preview.3,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,http://books.google.com/books?id=LZ77DwAAQBAJ&...,http://books.google.com/books?id=LZ77DwAAQBAJ&...,https://books.google.com/books/about/Legge_Di_...
1,Руководство по вирусному маркетингу с нуля,[Carolina Meli],Tektime,2020-07-23,Концепция вирусного маркетинга проста. Реализу...,"[{'type': 'ISBN_13', 'identifier': '9788835409...","{'text': True, 'image': True}",38,BOOK,[Business & Economics],NOT_MATURE,False,0.1.1.0.preview.3,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,http://books.google.com/books?id=wp7zDwAAQBAJ&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...
2,Курс делового английского / Business Issues,[Анна Крутских],Litres,2013-12-17,Пособие составлено с учетом требований Российс...,"[{'type': 'ISBN_13', 'identifier': '9785457520...","{'text': False, 'image': True}",177,BOOK,[Foreign Language Study],NOT_MATURE,False,0.1.1.0.preview.1,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,http://books.google.com/books?id=GiBcAgAAQBAJ&...,http://books.google.com/books?id=GiBcAgAAQBAJ&...,https://books.google.com/books/about/%D0%9A%D1...
3,Как иметь дело с людьми на работе,[Carolina Meli],Tektime,2020-07-29,"Пожалуй, очень многие люди (даже больше, чем в...","[{'type': 'ISBN_13', 'identifier': '9788835409...","{'text': True, 'image': True}",27,BOOK,[Business & Economics],NOT_MATURE,False,0.1.1.0.preview.3,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,http://books.google.com/books?id=yFn0DwAAQBAJ&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...
4,Привлекательный Маркетинг Для Начинающих,[Carolina Meli],Litres,2020-12-17,"Притягивайте людей к себе, а не гоняйтесь за н...","[{'type': 'ISBN_13', 'identifier': '9785043199...","{'text': True, 'image': True}",23,BOOK,[Business & Economics],NOT_MATURE,False,0.57.56.0.preview.3,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,http://books.google.com/books?id=KzgREAAAQBAJ&...,http://books.google.com/books?id=KzgREAAAQBAJ&...,https://books.google.com/books/about/%D0%9F%D1...


In [32]:
df_2.to_csv("more_bookshelf_books_api.csv", index=False)

Получения данных по определенному автору

In [37]:
nauthor = 'Shakespeare'

In [38]:
page_3 = requests.get(f'https://www.googleapis.com/books/v1/volumes?q=inauthor:{nauthor}', headers ={'accept': 'application/json', 'X-API-KEY': API_key})
page_3

<Response [200]>

In [39]:
res_3 = page_3.json()

In [42]:
res_3.keys()

dict_keys(['kind', 'totalItems', 'items'])

In [40]:
res_3

{'kind': 'books#volumes',
 'totalItems': 200,
 'items': [{'kind': 'books#volume',
   'id': 'nTpLDAAAQBAJ',
   'etag': 'uq+Wd+k6plc',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/nTpLDAAAQBAJ',
   'volumeInfo': {'title': 'Twelfth Night',
    'authors': ['William Shakespeare'],
    'publisher': 'Penguin',
    'publishedDate': '2016-07-05',
    'description': 'The acclaimed Pelican Shakespeare series, now in a dazzling new series design Winner of the 2016 AIGA + Design Observer 50 Books | 50 Covers competition This edition of Twelfth Night is edited with an introduction by Jonathan Crewe and was recently repackaged with cover art by Manuja Waldia. Waldia received a Gold Medal from the Society of Illustrators for the Pelican Shakespeare series. The legendary Pelican Shakespeare series features authoritative and meticulously researched texts paired with scholarship by renowned Shakespeareans. Each book includes an essay on the theatrical world of Shakespeare’s time, an introd

In [43]:
d_3 = res_3['items']

In [44]:
info_list = [item['volumeInfo'] for item in d_3]

In [45]:
df_3 = pd.DataFrame(info_list)
df_3

,title,authors,publisher,publishedDate,description,industryIdentifiers,readingModes,pageCount,printType,categories,maturityRating,allowAnonLogging,contentVersion,panelizationSummary,imageLinks,language,previewLink,infoLink,canonicalVolumeLink,subtitle
0,Twelfth Night,[William Shakespeare],Penguin,2016-07-05,"The acclaimed Pelican Shakespeare series, now ...","[{'type': 'ISBN_13', 'identifier': '9780143128...","{'text': False, 'image': False}",146,BOOK,[Drama],NOT_MATURE,False,0.3.1.0.preview.0,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=nTpLDAAAQBAJ&...,http://books.google.com/books?id=nTpLDAAAQBAJ&...,https://books.google.com/books/about/Twelfth_N...,NaN
1,The Merchant of Venice,[William Shakespeare],Hal Leonard Corporation,1998,(Applause Books). If there ever has been a gro...,"[{'type': 'ISBN_10', 'identifier': '1557832927...","{'text': False, 'image': True}",248,BOOK,[Drama],NOT_MATURE,False,2.5.4.0.preview.1,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=dQcg1sDSL9EC&...,http://books.google.com/books?id=dQcg1sDSL9EC&...,https://books.google.com/books/about/The_Merch...,NaN
2,The Tragedy of Hamlet,[William Shakespeare],Graphic Arts Books,2022-01-11,Hamlet (1601) is a tragedy by William Shakespe...,"[{'type': 'ISBN_13', 'identifier': '9781513210...","{'text': True, 'image': True}",114,BOOK,[Drama],NOT_MATURE,False,0.3.2.0.preview.3,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=NyJaEAAAQBAJ&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...,NaN
3,"Hamlet by William Shake-speare, 1603; Hamlet b...",[William Shakespeare],NaN,1860,NaN,"[{'type': 'OTHER', 'identifier': 'OXFORD:59090...","{'text': False, 'image': True}",244,BOOK,NaN,NOT_MATURE,False,0.3.6.0.full.1,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=9Z8NAAAAQAAJ&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...,NaN
4,As You Like It,[William Shakespeare],Graphic Arts Books,2022-01-11,As You Like It (1599) is a comedy by William S...,"[{'type': 'ISBN_13', 'identifier': '9781513210...","{'text': True, 'image': True}",83,BOOK,[Drama],NOT_MATURE,False,0.3.2.0.preview.3,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=MSJaEAAAQBAJ&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...,NaN
5,Othello,[William Shakespeare],e-artnow,2017-10-16,"""The Tragedy of Othello, the Moor of Venice"" i...","[{'type': 'ISBN_13', 'identifier': '9788027223...","{'text': True, 'image': True}",653,BOOK,[Biography & Autobiography],NOT_MATURE,True,1.3.4.0.preview.3,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=KWdODwAAQBAJ&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...,Including The Life of William Shakespeare
6,King Lear,[William Shakespeare],DigiCat,2022-05-25,King Lear is a tragedy by William Shakespeare ...,"[{'type': 'OTHER', 'identifier': 'EAN:85965470...","{'text': True, 'image': True}",629,BOOK,[Biography & Autobiography],NOT_MATURE,False,1.4.4.0.preview.3,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=V_dyEAAAQBAJ&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...,"Including ""The Life of William Shakespeare"""
7,Shakespeare's A Midsummer Night's Dream,[William Shakespeare],Courier Corporation,2012

In [46]:
df_3.to_csv("more_shakespeare_books_api.csv", index=False)

Запрос на получение книг определенной категории

In [53]:
categ = 'Fiction'

In [54]:
page_4 = requests.get(f'https://www.googleapis.com/books/v1/volumes?q=subject:{categ}', headers ={'accept': 'application/json', 'X-API-KEY': API_key})
page_4

<Response [200]>

In [55]:
res_4 = page_4.json()

In [56]:
res_4.keys()

dict_keys(['kind', 'totalItems', 'items'])

In [57]:
res_4

{'kind': 'books#volumes',
 'totalItems': 200,
 'items': [{'kind': 'books#volume',
   'id': 'FOHtDwAAQBAJ',
   'etag': '0E+vuYMYQpo',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/FOHtDwAAQBAJ',
   'volumeInfo': {'title': 'Ready Player Two',
    'subtitle': 'A Novel',
    'authors': ['Ernest Cline'],
    'publisher': 'Ballantine Books',
    'publishedDate': '2020-11-24',
    'description': '#1 NEW YORK TIMES BESTSELLER • The thrilling sequel to the beloved worldwide bestseller Ready Player One, the near-future adventure that inspired the blockbuster Steven Spielberg film. NAMED ONE OF THE BEST BOOKS OF THE YEAR BY THE WASHINGTON POST • “The game is on again. . . . A great mix of exciting fantasy and threatening fact.”—The Wall Street Journal AN UNEXPECTED QUEST. TWO WORLDS AT STAKE. ARE YOU READY? Days after winning OASIS founder James Halliday’s contest, Wade Watts makes a discovery that changes everything. Hidden within Halliday’s vaults, waiting for his heir to find, li

In [58]:
d_4 = res_4['items']

In [59]:
info_list = [item['volumeInfo'] for item in d_4]

In [60]:
df_4 = pd.DataFrame(info_list)
df_4

,title,subtitle,authors,publisher,publishedDate,description,industryIdentifiers,readingModes,pageCount,printType,...,ratingsCount,maturityRating,allowAnonLogging,contentVersion,panelizationSummary,imageLinks,language,previewLink,infoLink,canonicalVolumeLink
0,Ready Player Two,A Novel,[Ernest Cline],Ballantine Books,2020-11-24,#1 NEW YORK TIMES BESTSELLER • The thrilling s...,"[{'type': 'ISBN_13', 'identifier': '9781524761...","{'text': True, 'image': False}",385,BOOK,...,3.0,NOT_MATURE,True,3.15.14.0.preview.2,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=FOHtDwAAQBAJ&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...
1,If Beale Street Could Talk,NaN,[James Baldwin],Vintage,2013-09-17,From one of the most important writers of the ...,"[{'type': 'ISBN_13', 'identifier': '9780804149...","{'text': True, 'image': False}",210,BOOK,...,NaN,NOT_MATURE,True,2.5.3.0.preview.2,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=lclWAAAAQBAJ&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...
2,There There,A novel,[Tommy Orange],Vintage,2018-06-05,PULITZER PRIZE FINALIST • NATIONAL BESTSELLER ...,"[{'type': 'ISBN_13', 'identifier': '9780525520...","{'text': True, 'image': False}",260,BOOK,...,2.0,NOT_MATURE,True,1.18.16.0.preview.2,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=oNY0DwAAQBAJ&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...
3,The Plague,NaN,[Albert Camus],Vintage,1991-05-07,“Its relevance lashes you across the face.” —S...,"[{'type': 'ISBN_13', 'identifier': '9780679720...","{'text': False, 'image': False}",312,BOOK,...,14.0,NOT_MATURE,False,1.1.1.0.preview.0,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=KVGd-NabpW0C&...,http://books.google.com/books?id=KVGd-NabpW0C&...,https://books.google.com/books/about/The_Plagu...
4,Bag of Bones,NaN,[Stephen King],Pocket Books,1999-06-01,From #1 New York Times bestselling author Step...,"[{'type': 'ISBN_10', 'identifier': '067102423X...","{'text': False, 'image': False}",756,BOOK,...,3.0,NOT_MATURE,False,0.5.2.0.preview.0,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=FxmjXM5RNNwC&...,http://books.google.com/books?id=FxmjXM5RNNwC&...,https://books.google.com/books/about/Bag_of_Bo...
5,The Bean Trees,A Novel,[Barbara Kingsolver],Harper Collins,1998-09-09,"Clear-eyed and spirited, Taylor Greer grew up ...","[{'type': 'ISBN_10', 'identifier': '0061097314...","{'text': False, 'image': False}",340,BOOK,...,23.0,NOT_MATURE,False,1.4.5.0.preview.0,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=O-XiVJO1K1gC&...,http://books.google.com/books?id=O-XiVJO1K1gC&...,https://books.google.com/books/about/The_Bean_...
6,My Name Is Asher Lev,NaN,[Chaim Potok],Anchor,2009-07-01,NATIONAL BESTSELLER • In this modern classic f...,"[{'type': 'ISBN_13', 'identifier': '9780307422...","{'text': True, 'image': True}",385,BOOK,...,4.0,NOT_MATURE,True,3.11.10.0.preview.3,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com/books?id=kYcUeOR4bdoC&...,https://play.google.com/store/books/details?id...,https://play.google.com/store/books/details?id...
7,The Twelve Chairs,NaN,"[Ilʹi︠a︡ Ilʹf, Evgeniĭ Petrov]",Northwestern University Press,1997,Ostap Bender is an unemployed con artist livin...,"[{'type': 'ISBN_10', 'identifier': '0810114844...","{'text': Tr

In [61]:
df_4.to_csv("more_fiction_books_api.csv", index=False)